In [49]:
import torch
import torch.nn as nn
from transformer import *
from config import get_weights_file_path
from train import *
from dataset import *
import altair as alt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [50]:
def get_device():
    #check for cpu, gpu and mps
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    print(f"Using device: {device}")
    return device

device = get_device()

Using device: mps


In [51]:
import os
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_data_set(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config, f"19")
print(f"Loading model from {model_filename}")
if not os.path.exists(model_filename):
    raise FileNotFoundError(f"Model file {model_filename} does not exist.")
state = torch.load(model_filename)
model.load_state_dict(state['model'])


Building tokenizer for en...
Dataset size: 32332 sentences
Dataset columns: ['id', 'translation']
Example sentences in en:
1: Source: Project Gutenberg
2: Jane Eyre
3: Charlotte Bronte
4: CHAPTER I
5: There was no possibility of taking a walk that day.
Tokenizer path: tokenizer_en.json
Building tokenizer for it...
Dataset size: 32332 sentences
Dataset columns: ['id', 'translation']
Example sentences in it:
1: Source: www.liberliber.it/Audiobook available here
2: Jane Eyre
3: Charlotte Brontë
4: PARTE PRIMA
5: I. In quel giorno era impossibile passeggiare.
Tokenizer path: tokenizer_it.json
Max length source: 309, Max length target: 274
Loading model from weights/tmodel_19.pt


<All keys matched successfully>

In [52]:
def load_next_batch():
     # Load a sample batch from the validation set
    batch = next(iter(val_dataloader))
    encoder_input = batch["encoder_input"].to(device)
    encoder_mask = batch["encoder_mask"].to(device)
    decoder_input = batch["decoder_input"].to(device)
    decoder_mask = batch["decoder_mask"].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]

    # check that the batch size is 1
    assert encoder_input.size(
        0) == 1, "Batch size must be 1 for validation"

    model_out = greedy_decode(
        model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config['seqlen'], device)
    
    return batch, encoder_input_tokens, decoder_input_tokens

def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )
def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [53]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
sentence_len = encoder_input_tokens.index("[PAD]")

Source: Levin did not refuse the proffered wine, and ordered another bottle.
Target: Levin non rifiutò il vino offertogli e chiese un’altra bottiglia.


In [ ]:
layers = [0, 1, 2]
heads = [0, 1, 2, 3, 4, 5, 6, 7]

# Encoder Self-Attention
get_all_attention_maps("encoder", layers, heads, encoder_input_tokens, encoder_input_tokens, min(20, sentence_len))